![clothing_classification](clothing_classification.png)


Fashion Forward is a new AI-based e-commerce clothing retailer.
They want to use image classification to automatically categorize new product listings, making it easier for customers to find what they're looking for. It will also assist in inventory management by quickly sorting items.

As a data scientist tasked with implementing a garment classifier, your primary objective is to develop a machine learning model capable of accurately categorizing images of clothing items into distinct garment types such as shirts, trousers, shoes, etc.

Automate product tagging for the e-commerce store using CNNs.

Once trained (keeping the epochs to 1 or 2 to keep the run time down), store your predictions on the test set in a list named predictions.

Calculate the accuracy, and per-class precision and recall for your classifier based on the predictions obtained. Store your metrics in variables named accuracy, precision, and recall. Use lists of the appropriate length for the precision and recall.

In [28]:
# Run the cells below first

In [29]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall

In [30]:
# Load datasets
from torchvision import datasets
import torchvision.transforms as transforms

train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

In [31]:
# Start coding here
# Use as many cells as you need

In [32]:
# Get the number of classes
classes = train_data.classes
num_classes = len(train_data.classes)

# Define some relevant variables
num_input_channels = 1
num_output_channels = 16
image_size = train_data[0][0].shape[1]

In [33]:
# 1. Defining the CNN (Convolutional Neural Network)
# Define CNN
class MultiClassImageClassifier(nn.Module): # Creating a class to contain the layers of a CNN
    def __init__(self):
        super(MultiClassImageClassifier, self).__init__()
        self.conv = nn.Conv2d(in_channels = num_input_channels, out_channels = num_output_channels, kernel_size=3, stride=1, padding=1) # Adding a convolutional layer
        self.relu = nn.ReLU() # Adding a Rectilinear Unit
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2) # Adding a pooling layer
        self.flatten = nn.Flatten()  # flatten the input

        # Create a fully connected layer
        self.fc = nn.Linear(num_output_channels * (image_size//2)**2, num_classes) # Adding a fully connected layer

    def forward(self, x): # Defining a .forward() method
        # Pass inputs through each layer
        x = self.conv(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x


In [34]:
# 2. Training the CNN

# Define the training set DataLoader
dataloader_train = DataLoader(
    train_data,
    batch_size=10,
    shuffle=True,
)

# Define training function
def train_model(optimizer, net, num_epochs):
    num_processed = 0
    criterion = nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        running_loss = 0
        num_processed = 0
        for features, labels in dataloader_train:
            optimizer.zero_grad()
            outputs = net(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            num_processed += len(labels)
        print(f'epoch {epoch}, loss: {running_loss / num_processed}')
        
    train_loss = running_loss / len(dataloader_train)

# Train for 1 epoch
net = MultiClassImageClassifier()  # <-- FIXED: removed num_classes argument
optimizer = optim.Adam(net.parameters(), lr=0.001)

train_model(
    optimizer=optimizer,
    net=net,
    num_epochs=1,
)

epoch 0, loss: 0.03969368415138645


In [35]:
# 3. Testing the CNN


# Test the model on the test set
              
# Define the test set DataLoader
dataloader_test = DataLoader(
    test_data,
    batch_size=10,
    shuffle=False,
)
# Define the metrics
accuracy_metric = Accuracy(task='multiclass', num_classes=num_classes)
precision_metric = Precision(task='multiclass', num_classes=num_classes, average=None)
recall_metric = Recall(task='multiclass', num_classes=num_classes, average=None)

# Run model on test set
net.eval()
predictions = []
for i, (features, labels) in enumerate(dataloader_test):
    output = net.forward(features.reshape(-1, 1, image_size, image_size))
    cat = torch.argmax(output, dim=-1)
    predictions.extend(cat.tolist())
    accuracy_metric(cat, labels)
    precision_metric(cat, labels)
    recall_metric(cat, labels)

# Compute the metrics
accuracy = accuracy_metric.compute().item()
precision = precision_metric.compute().tolist()
recall = recall_metric.compute().tolist()
print('Accuracy:', accuracy)
print('Precision (per class):', precision)
print('Recall (per class):', recall)

Accuracy: 0.8779000043869019
Precision (per class): [0.8836363554000854, 0.9978991746902466, 0.818092405796051, 0.7851170301437378, 0.8211716413497925, 0.9632571935653687, 0.6676245331764221, 0.9438315033912659, 0.9756592512130737, 0.9571285843849182]
Recall (per class): [0.7289999723434448, 0.949999988079071, 0.8320000171661377, 0.9390000104904175, 0.7990000247955322, 0.9700000286102295, 0.6970000267028809, 0.9409999847412109, 0.9620000123977661, 0.9599999785423279]


Guid

How to approach the project
1. Defining the CNN (Convolutional Neural Network)

2. Training the CNN

3. Testing the CNN

Steps to complete

1
Defining the CNN (Convolutional Neural Network)
Define a class containing all the appropriate layers, and a method to perform the forward pass of a batch of images.


Creating a class to contain the layers of a CNN
You could define a class that inherits from PyTorch's nn.module class.



Adding a convolutional layer
You could use PyTorch's nn.Conv2D() class to define the convolutional layer.
Create an instance of it in your CNN class's constructor and assign it to an instance variable such as self.conv.



Adding a Rectilinear Unit
You could use PyTorch's nn.ReLU() class.
Create an instance of it in your CNN class's constructor and assign it to an instance variable such as self.relu.



Adding a pooling layer
You could use PyTorch's nn.MaxPool2D() class.
Create an instance of it in your CNN class's constructor and assign it to an instance variable such as self.maxpool.



Adding a fully connected layer
You could use PyTorch's nn.Linear() class.
Create an instance of it in your CNN class's constructor and assign it to an instance variable such as self.fc.
You will also need to flatten the input first, which could be done with an instance of nn.Flatten



Defining a .forward() method
Finally, you'll need to define a .forward() method that passes the input through each layer and returns the output.



2
Training the CNN
Define a training loop that loops over the dataset, calculating the loss and propagating it backwards through the network.


Define a suitable loss criterion
PyTorch's nn.CrossEntropyLoss() could be used here, since this is a multi-class classification problem.



Define an optimizer
You could use PyTorch's optim.Adam() optimizer here.



3
Testing the CNN
Use your trained model to classify the images in the test set, and calculate the appropriate metrics.


Predict the category of each image in the test data.
You'll need to use the .forward() method on your CNN class to pass the test images through the network.
You could use torch.argmax() to find the category with the highest predicted probability.



Calculate the performance metrics
You could use Accuracy(), Precision(), and Recall() from torchmetrics to calculate the

In [ ]:
#### Solution ####

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall

# Dataset loading not copied here, please refer to the notebook for this

# Get the number of classes
classes = train_data.classes
num_classes = len(train_data.classes)

# Define some relevant variables
num_input_channels = 1
num_output_channels = 16
image_size = train_data[0][0].shape[1]

# Define CNN
class MultiClassImageClassifier(nn.Module):
  
    # Define the init method
    def __init__(self, num_classes):
        super(MultiClassImageClassifier, self).__init__()
        self.conv1 = nn.Conv2d(num_input_channels, num_output_channels, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

        # Create a fully connected layer
        self.fc = nn.Linear(num_output_channels * (image_size//2)**2, num_classes)
        
    def forward(self, x):
        # Pass inputs through each layer
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x
      
# Define the training set DataLoader
dataloader_train = DataLoader(
    train_data,
    batch_size=10,
    shuffle=True,
)

# Define training function
def train_model(optimizer, net, num_epochs):
    num_processed = 0
    criterion = nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        running_loss = 0
        num_processed = 0
        for features, labels in dataloader_train:
            optimizer.zero_grad()
            outputs = net(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            num_processed += len(labels)
        print(f'epoch {epoch}, loss: {running_loss / num_processed}')
        
    train_loss = running_loss / len(dataloader_train)


# Train for 1 epoch
net = MultiClassImageClassifier(num_classes)
optimizer = optim.Adam(net.parameters(), lr=0.001)

train_model(
    optimizer=optimizer,
    net=net,
    num_epochs=1,
)

# Test the model on the test set
              
# Define the test set DataLoader
dataloader_test = DataLoader(
    test_data,
    batch_size=10,
    shuffle=False,
)
# Define the metrics
accuracy_metric = Accuracy(task='multiclass', num_classes=num_classes)
precision_metric = Precision(task='multiclass', num_classes=num_classes, average=None)
recall_metric = Recall(task='multiclass', num_classes=num_classes, average=None)

# Run model on test set
net.eval()
predictions = []
for i, (features, labels) in enumerate(dataloader_test):
    output = net.forward(features.reshape(-1, 1, image_size, image_size))
    cat = torch.argmax(output, dim=-1)
    predictions.extend(cat.tolist())
    accuracy_metric(cat, labels)
    precision_metric(cat, labels)
    recall_metric(cat, labels)

# Compute the metrics
accuracy = accuracy_metric.compute().item()
precision = precision_metric.compute().tolist()
recall = recall_metric.compute().tolist()
print('Accuracy:', accuracy)
print('Precision (per class):', precision)
print('Recall (per class):', recall)